In [ ]:
mensaje_system = {
    "role": "system",
    "content": (
        f"Eres un asistente en ingeniería eléctrica que analiza comentarios históricos de analistas "
        f"y alarmas de medidores de energía. Basándote en los comentarios y las alarmas recibidas, "
        f"determina si se requiere una orden de servicio y clasifica la criticidad del problema. "
        f"Proporciona una justificación adecuada en tus respuestas."
    )
}

# Crear y escribir en el archivo JSONL
with open('fine_tuning_data_hexing_v2.jsonl', 'w', encoding='utf-8') as archivo:
    for index, row in df.iterrows():
        # Extraer alarmas relevantes (ejemplo de cómo se podría estructurar)
        alarmas = [
            {"name": "Watchdog Error/Exception Reset", "explanation": "Indica un reinicio del perro guardián o un reinicio del hardware del microcontrolador."},
            {"name": "Voltage Unbalance Start", "explanation": "La tensión trifásica está desequilibrada."},
            # Agregar más alarmas según corresponda
        ]
        
        # Convertir las alarmas en texto legible
        alarmas_texto = "\n".join([f"{alarma['name']}: {alarma['explanation']}" for alarma in alarmas])

        # Crear estructura de mensaje para el role "user"
        mensajes = [
            mensaje_system,
            {
                "role": "user",
                "content": (
                    f"¿Requiere la clave {row['CLAVE']} con el ticket {row['TICKET']} del circuito {row['CIRCUITO']}, "
                    f"y fecha de análisis {row['FECHA_ANALISIS']}, <<con la descripción  {row['COMENTARIO_ANALISTA']}>> "
                    f"una orden de servicio? Las alarmas detectadas son:\n{alarmas_texto}\n"
                    f"¿Es recurrente este problema? ¿Cuál es la no conformidad?"
                )
            },
            {
                "role": "assistant",
                "content": (
                    f"Reporte de Evaluación:\n"
                    f"Ticket: {row['TICKET']}\n"
                    f"Clave: {row['CLAVE']}\n"
                    f"Circuito: {row['CIRCUITO']}\n"
                    f"Requiere Orden de Servicio: {'Sí' if row['REQUIERE_OS'] == 1 else 'No'}\n"
                    f"No Conformidad: {row['NO_CONFORMIDAD']}\n"
                    f"Es Recurrente: {'Sí' if row['ES_RECURRENTE'] == 1 else 'No'}\n"
                )
            }
        ]
        
        # Escribir el conjunto de mensajes en una estructura JSON
        archivo.write(json.dumps({"messages": mensajes}, ensure_ascii=False) + '\n')


In [2]:
#cargar txt
import pandas as pd
import json

# cargar el archivo .txt

with open('alarmas_hexing.txt', 'r', encoding='utf-8') as archivo:
    alarmas = archivo.read().splitlines()

mensaje_system = {
    "role": "system",
    "content": (
        f"Eres un especialista en detección de anomalías para medidores de energía. Explica el significado de cada alarma."
    )
}

#eliminar fila 0
alarmas.pop(0)

#crear jsonl

with open('alarmas_hexing.jsonl', 'w', encoding='utf-8') as archivo:
    # recorrer
    for alarma in alarmas:
        data = alarma.split('/salto/')
        mensajes = [
            mensaje_system,
            {
                "role": "user",
                "content": (
                    f"¿Qué implica la alarma {data[0]}?"	
                
                )
            },
            {
                "role": "assistant",
                "content": (
                f"La alarma {data[0]} indica {data[2]}"

                )
            }
        ]
        archivo.write(json.dumps({"messages": mensajes}, ensure_ascii=False) + '\n')

In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import json
import os
from dotenv import load_dotenv

load_dotenv()

# Establecer la conexión a la base de datos
try:
    connection = mysql.connector.connect(
        host=os.getenv('HOST'),
        user=os.getenv('USERNAME'),
        password=os.getenv('PASSWORD'),
        database=os.getenv('DATABASE')
    )

    if connection.is_connected():
        print("Conexión a MySQL exitosa")

        # Ejecutar la consulta y cargar los resultados en un DataFrame de pandas
        consulta_sql = """
            SELECT TICKET, CLAVE, FECHA_ANALISIS, NO_CONFORMIDAD, REQUIERE_OS, COMENTARIO_ANALISTA, ES_RECURRENTE, MERCADO, CONSUMO_PROMEDIO, MEDIDA, CIRCUITO
            FROM pnrp.bitacora_ac
            WHERE MARCA = 'HEXING'
            AND ESTADO = 'COMPLETADO'
            AND CLAVE IN (
                SELECT CLAVE
                FROM pnrp.bitacora_ac
                WHERE MARCA = 'HEXING' AND ESTADO = 'COMPLETADO'
                AND REQUIERE_OS = 1
                GROUP BY CLAVE
                HAVING COUNT(*) > 2
            );
        """

        # Leer los datos usando pandas
        df = pd.read_sql(consulta_sql, connection)

        # Definir el "system" role único fuera del bucle
        mensaje_system = {
            "role": "system",
            "content": (
                f"Eres un asistente en ingeniería eléctrica que, basándote en comentarios históricos "
                f"de analistas que pueden incluir algunos datos como el número de ticket, la clave del dispositivo, "
                f"la fecha de análisis, la no conformidad identificada, si requiere o no una orden de servicio, "
                f"el comentario del analista, si es recurrente, el mercado al que pertenece, el consumo promedio, "
                f"el tipo de medida, y el circuito, intenta correlacionar e inferir si requiere o no una orden de servicio así como la clasificación de una criticidad, proporciona una justificación "
                f"adecuada en tus respuestas."
            )
        }

        # Crear y escribir en el archivo JSONL
        with open('fine_tuning_data_hexing_si_v1.jsonl', 'w', encoding='utf-8') as archivo:
            for index, row in df.iterrows():
                # Crear estructura de mensaje para el role "user"
                mensajes = [
                    mensaje_system,
                    {
                        "role": "user",
                        "content": (
                            f"¿Requiere la clave {row['CLAVE']} con el ticket {row['TICKET']} del circuito {row['CIRCUITO']}, "
                            f"y fecha de análisis {row['FECHA_ANALISIS']}, <<con la descripción  {row['COMENTARIO_ANALISTA']}>> "
                            f"una orden de servicio? ¿Es recurrente este problema? ¿Cuál es la no conformidad?"
                        )
                    },
                    {
                        "role": "assistant",
                        "content": (
                            f"Reporte de Evaluación:\n"
                            f"Ticket: {row['TICKET']}\n"
                            f"Clave: {row['CLAVE']}\n"
                            f"Circuito: {row['CIRCUITO']}\n"
                            f"Requiere Orden de Servicio: {'Sí' if row['REQUIERE_OS'] == 1 else 'No'}\n"
                            f"No Conformidad: {row['NO_CONFORMIDAD']}\n"
                            f"Es Recurrente: {'Sí' if row['ES_RECURRENTE'] == 1 else 'No'}\n"
                        )
                    }
                ]
                
                # Escribir el conjunto de mensajes en una estructura JSON
                archivo.write(json.dumps({"messages": mensajes}, ensure_ascii=False) + '\n')

        print("Archivo JSONL generado exitosamente.")

except Error as e:
    print(f"Error al conectar con MySQL: {e}")

finally:
    if connection.is_connected():
        connection.close()
        print("Conexión a MySQL cerrada")